NOTE:
You are supposeed to launch moveit demo. 
Also, change "group_name" correspoinding to your robot's planning name.

In [1]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg
from math import pi
from std_msgs.msg import String
from moveit_commander.conversions import pose_to_list

In [2]:
def all_close(goal, actual, tolerance):
  """
  Convenience method for testing if a list of values are within a tolerance of their counterparts in another list
  @param: goal       A list of floats, a Pose or a PoseStamped
  @param: actual     A list of floats, a Pose or a PoseStamped
  @param: tolerance  A float
  @returns: bool
  """
  all_equal = True
  if type(goal) is list:
    for index in range(len(goal)):
      if abs(actual[index] - goal[index]) > tolerance:
        return False

  elif type(goal) is geometry_msgs.msg.PoseStamped:
    return all_close(goal.pose, actual.pose, tolerance)

  elif type(goal) is geometry_msgs.msg.Pose:
    return all_close(pose_to_list(goal), pose_to_list(actual), tolerance)

  return True

In [3]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('move_group_python_interface_tutorial',anonymous=True)
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
## set robot name ##
group_name = "hc10_arm"
group = moveit_commander.MoveGroupCommander(group_name)
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path',
                                                   moveit_msgs.msg.DisplayTrajectory,
                                                   queue_size=20)

RuntimeError: RobotInterfacePython: invalid robot model

In [ ]:
print "============ Printing robot state"
print robot.get_current_state()
print ""

In [12]:
### joint control

## Get current joint states
joint_goal = group.get_current_joint_values()
## Decreased joint_goal[0] by 0.1 rad, which means joint_1_s
joint_goal[1] = joint_goal[1] + 0.1
group.go(joint_goal, wait=True)
group.stop()
current_joints = group.get_current_joint_values()
all_close(joint_goal, current_joints, 0.01)

False

In [13]:
### Cartesian control

pose_goal = group.get_current_pose().pose
print pose_goal.position
pose_goal.position.z = pose_goal.position.z - 0.1
group.set_pose_target(pose_goal)
plan = group.go(wait=True)
group.stop()
group.clear_pose_targets()
current_pose = group.get_current_pose().pose
all_close(pose_goal, current_pose, 0.01)

x: 0.686570421042
y: 0.180566317577
z: 1.18885809075


False

In [14]:
### Change maximum speeds of all joints [rad/s] with scalling factor
velocity_scale_factor = 1
group.set_max_velocity_scaling_factor(velocity_scale_factor)
### Change maximum accerelation of all joints [rad/s] with scalling factor
accerelation_scale_factor = 1
group.set_max_acceleration_scaling_factor(accerelation_scale_factor)

In [8]:
pose_goal = group.get_current_pose().pose
print pose_goal

position: 
  x: -1.0007157552
  y: -0.391880869492
  z: 1.3863149243
orientation: 
  x: 0.0462167653147
  y: 0.708960420459
  z: -0.700973741634
  w: 0.0622490671896


In [16]:
### Cartesian control

pose_goal = group.get_current_pose().pose
print pose_goal.position
pose_goal.position.z = pose_goal.position.z + 0.1
group.set_pose_target(pose_goal)
plan = group.go(wait=True)
group.stop()
group.clear_pose_targets()
current_pose = group.get_current_pose().pose
all_close(pose_goal, current_pose, 0.01)

x: 0.686570421042
y: 0.180566317577
z: 1.18885809075


False

In [17]:
### Cartesian contro; with waypoints
waypoints = []
scale = 1
wpose = group.get_current_pose().pose
wpose.position.z -= scale * 0.2  # First move up (z)
wpose.position.y += scale * 0.4  # and sideways (y)
waypoints.append(copy.deepcopy(wpose))

wpose.position.x += scale * 0.2  # Second move forward/backwards in (x)
waypoints.append(copy.deepcopy(wpose))

wpose.position.y -= scale * 0.2  # Third move sideways (y)
waypoints.append(copy.deepcopy(wpose))
    
    
(plan, fraction) = group.compute_cartesian_path(
                                    waypoints,   # waypoints to follow
                                    0.01,        # eef_step
                                    0.0)         # jump_threshold

In [18]:
### Display trajectory
display_trajectory = moveit_msgs.msg.DisplayTrajectory()
display_trajectory.trajectory_start = robot.get_current_state()
display_trajectory.trajectory.append(plan)
# Publish
display_trajectory_publisher.publish(display_trajectory);

In [19]:
### Execute the trajectory with waypoints
group.execute(plan, wait=True)

False